In [1]:
## Importing the relevant packages:

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import math
from statistics import mean
import pandas as pd
import numpy as np
from nltk.tokenize import word_tokenize
from nltk import pos_tag
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.preprocessing import LabelEncoder
from collections import defaultdict
from nltk.corpus import wordnet as wn
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn import model_selection, naive_bayes, svm
from sklearn.metrics import accuracy_score
import nltk


KeyboardInterrupt



In [ ]:
import nltk
nltk.download('averaged_perceptron_tagger')

In [ ]:
data_df = pd.read_csv("consumer_survey_data.csv")
print(len(data_df))
data_df.head()

In [ ]:
df = data_df[['group', 'survey_purpose']]
print(len(df))

df = df[(df['group'] == "T1") | ((df['group'] == "control"))]
df['group'].value_counts()

In [ ]:
def get_label(col):
    if ((col == "T1") | (col == "T2") | (col == "T3") | (col == "T4")):
        return 1
    else:
        return 0
    
df['label'] = df['group'].apply(get_label)
df['label'].value_counts()

In [ ]:
df.head()

In [ ]:
df['text'] = df['survey_purpose']
df['text'].dropna(inplace = True)
df['text'] = df['text'].astype(str)
df['text'] = [entry.lower() for entry in df['text']]
df['text']= [word_tokenize(entry) for entry in df['text']]

print(len(df))
df = df.reset_index()
df.head()

In [ ]:
tag_map = defaultdict(lambda : wn.NOUN)
tag_map['J'] = wn.ADJ
tag_map['V'] = wn.VERB
tag_map['R'] = wn.ADV

for index, entry in enumerate(df['text']):
    Final_words = []
    word_Lemmatized = WordNetLemmatizer()    
    for word, tag in pos_tag(entry):
        if word not in stopwords.words('english') and word.isalpha():
            word_Final = word_Lemmatized.lemmatize(word, tag_map[tag[0]])
            Final_words.append(word_Final)
    df.loc[index, 'text_final'] = str(Final_words)
    
print(len(df))
df['text_final'] = df['text_final'].astype(str)
df.head()

In [ ]:
Train_X, Test_X, Train_Y, Test_Y = model_selection.train_test_split(df['text_final'], df['label'], test_size = 0.25)

Encoder = LabelEncoder()
Train_Y = Encoder.fit_transform(Train_Y)
Test_Y = Encoder.fit_transform(Test_Y)

print(len(Train_X), len(Test_X), len(Train_X) + len(Test_X))
print(len(Train_Y), len(Test_Y), len(Train_Y) + len(Test_Y))

Tfidf_vect = TfidfVectorizer(max_features = 5000)
Tfidf_vect.fit(df['text_final'])

Train_X_Tfidf = Tfidf_vect.transform(Train_X)
Test_X_Tfidf = Tfidf_vect.transform(Test_X)

Test_X_Tfidf 

In [ ]:
# Classifier - Algorithm - SVM
# Fit the training dataset on the classifier
SVM = svm.SVC(C = 1.0, kernel = 'linear', degree = 3, gamma = 'auto')
SVM.fit(Train_X_Tfidf, Train_Y)

# Predict the labels on validation dataset
predictions_SVM = SVM.predict(Test_X_Tfidf)

# Use accuracy_score function to get the accuracy
print("SVM Accuracy Score -> ", accuracy_score(predictions_SVM, Test_Y) * 100)

print(len(predictions_SVM))
#predictions_SVM

In [ ]:
test_df = pd.DataFrame(Test_X)
test_df = test_df.reset_index()
test_df

test_df['predicted_label'] = predictions_SVM
test_df['actual_label'] = Test_Y
print(test_df['actual_label'].sum(), test_df['predicted_label'].sum())
test_df

In [ ]:
def get_categories(df):
    if ((df['actual_label'] == 1) & (df['predicted_label'] == 1)):
        return "True Treated & Predicted Treated"
    
    elif ((df['actual_label'] == 1) & (df['predicted_label'] == 0)):
        return "True Treated & Predicted Control"
    
    elif ((df['actual_label'] == 0) & (df['predicted_label'] == 1)):
        return "True Control & Predicted Treated"
    
    elif ((df['actual_label'] == 0) & (df['predicted_label'] == 0)):
        return "True Control & Predicted Control"
    
    else:
        return 0
    
test_df['categories'] = test_df.apply(get_categories, axis = 1)
test_df['categories'].value_counts()